<a href="https://colab.research.google.com/github/jwosborne-knox/cudaBlur/blob/main/Midi_Player_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

To do this lab, you need to go into the Runtime menu, select "Change runtime type" and select a GPU hardware accelerator.

In addition, run the following cell to install some necessary functionality.

In [ ]:
# Install AnyCmd module, for specifying more detailed commands to be run upon a cell (https://github.com/zenarcher007/AnyCmd)
!pip3 install anycmd-jupyter-magic
%load_ext anycmd
# Note: please ignore the warning about needing to restart your runtime.

In addition, you need to upload the given code MIDI_lab.zip for this lab; select the file button to the left, then the upload button (up arrow inside a document).  Then, run the following code cell to open it:

In [ ]:
!unzip "MIDI_lab.zip" && rm "MIDI_lab.zip"

Next, we need the MIDI file that we're going to convert.  Download a copy of Beethoven's Fifth Symphony using this link:

https://kunstderfuge.com/-/midi.asp?file=beethoven/symphony_5_1_(c)cvikl.mid

It's to a subscription site (http://kunstderfuge.com/), but you are allowed 5 downloads a day without even registering.

Once you've downloaded that file, upload it to the notebook following the same procedure as above.

# MIDI to WAV conversion

Now, we're ready for the actual lab.  The next cell contains serial code to convert a MIDI file (controlled by the variable midFile, but currently set to symphony_5_1_(c)cvikl.mid) into a WAV file (out.wav).  Run it and then use the next cell to play the output file.

In [ ]:
%%any -l -p -i -d. -- nvcc -O3 %FILE.cu MidiFile/*.cpp -lstdc++fs -o file && ./file
// Sequential Version of MIDI converter
//
//  Originally created by Justin Douty on 7/8/22.

#define TUNING 264 // An arbitrary number specifying the "tuning" of the virtual instruments. Use this to tune the music to the correct pitch.
#define DECAY 0.45 // The decay of the notes after they stop playing, in seconds.

#include <iostream>
#include <experimental/filesystem>
#include <cmath>
#include <string.h>
#include "MidiFile/MidiFile.h" // Midi parsing library https://github.com/craigsapp/midifile
#include "WavHeader.h"
#include "InstLoader.h"

using namespace std;
using namespace smf; // Midi library namespace

int main(int argc, const char * argv[]) {
  /* Specify the files of instruments to use here, in order of their channels... */
  string instDataNames[16] = {"Flute.wav", "Oboe.wav", "Clarinet.wav", "Bassoon.wav", "FrenchHorn.wav", "Trumpet.wav", "Timpani.wav", "Cello.wav", "Cello.wav", "Cello.wav", "Cello.wav", "Strings.wav", "Violin.wav", "Strings.wav", "Cello.wav", "Contrabass.wav"};
  string instDir = "InstrumentData"; // The name of the instruments base directory
  string midFile = "symphony_5_1_(c)cvikl.mid"; // Specify the name of the MIDI file to play here
  
  
  InstLoader lab(instDir, instDataNames); // Read and allocate memory for instrument sound data
  cout << "Instrument data uses " << ((lab.getInstDataSize()*sizeof(float)) / 1024) / 1024 << " MiB\n";
  
  MidiFile mFile; // Midi library
  if(! mFile.read(midFile)) {
    cerr << "Error opening input MIDI file!\n";
    exit(1);
  }
  mFile.joinTracks(); // Combine all tracks into a single event stream
  mFile.doTimeAnalysis(); // Analyze absolute starting time, in seconds, of each note.
  mFile.linkNotePairs(); // Analyze duration of each note.
  
  int trackSize = mFile[0].size();
  
  int totalSteps = mFile.getFileDurationInSeconds() * 44100 + DECAY*44100;
  float* audioData = new float[totalSteps]; // Allocate a large portion of memory for the resulting audio data
  memset((void*) audioData, 0, totalSteps * sizeof(float)); // Clear this allocation
  
  auto start_time = std::chrono::high_resolution_clock::now(); // Measure start time
  
  for(int event = 0; event < trackSize; ++event) {
    MidiEvent* e = &mFile[0][event]; // Current MIDI event.
    if(! e->isNoteOn()) continue; // We are only concerned with note-on events; the library provides us with the length of the note.
    
    InstSection instSection = lab.getInstSection(e->getChannel()); // Retrieve instrument data of the specified channel
    float pitchMultiplier = (440.0f*pow(2.0f, ((e->getKeyNumber()-69)/12.0f))) / (float) TUNING; // Converts MIDI note numbers into a pitch stretch factor.
    float velocityMultiplier = e->getVelocity() / 127.0; // Convert 0-127 velocity into 0-1
    
    // start < end < decayEnd
    unsigned long start = e->seconds * 44100; // Start offset in resulting audio
    
    unsigned long idealEndPos = (unsigned long) (start + e->getDurationInSeconds() * 44100); // The ideal length of the note
    unsigned long actualEndPos = (unsigned long) (start + instSection.size / pitchMultiplier); // Length limited according to end of instSection
    unsigned long end = min(idealEndPos, actualEndPos); // End offset in resulting audio for main portion of note
    
    // End offset for note decay, same limiting schemantics as for "end"
    unsigned long decayEnd = min( (unsigned long) (end + DECAY*44100), (unsigned long) (start + instSection.size / pitchMultiplier));
    
    /* Render Note */
    for(unsigned long x = start; x < end; x++) {
      unsigned long instOffset = (unsigned long) ((x-start) * pitchMultiplier);
      audioData[x] += instSection.data[instOffset] * velocityMultiplier;
    }
    
    /* Render Note Decay */
    unsigned long noteSize = decayEnd - end;
    for(unsigned long x = end; x < decayEnd; x++) {
      unsigned long instOffset = (x - start) * pitchMultiplier;
      float releaseVol = (float) (decayEnd - x) / (float) noteSize;
      audioData[x] += instSection.data[instOffset] * velocityMultiplier * releaseVol;
    }
  }
  
  auto end_time = std::chrono::high_resolution_clock::now();
  cerr << "EXECUTION TIME: " << std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time).count() << " milliseconds\n";
  
  InstLoader::writeWaveFile(audioData, totalSteps, "out.wav"); // Write data to a WAVE file
}

In [ ]:
# Run this cell to play a portion of the generated audio file.
import numpy as np
import IPython.display as ipd

# You can set the start and end percentages of the song to play, or download the file from Colab and listen
start = 0
end = 5

a = np.fromfile("out.wav", dtype=np.float32, offset=90+4*start, sep="") # Read data from output file
startPos = int( 90 + (start / 100) * (len(a)-90) )
endPos = int( 90 + (end / 100) * (len(a)-90) )

ipd.Audio(a[startPos : endPos], rate=44100, autoplay = True) # Download and play audio

Now you're ready to "CUDAify" the translation process.  The cell below is the beginning of this.  The kernel processNote is called once for each note.  It should act as a replacement for the paragraphs of code marked "Render Note" and "Render Note Decay" in the serial version above.

When you are ready, run your code and play the resulting file (out_cuda.wav) using the bottom cell.

In [ ]:
%%any -l -p -i -d. -- nvcc -O3 %FILE.cu MidiFile/*.cpp -lstdc++fs -o file && ./file | head -n 10
//  Beginning of CUDA version
//
//  Originally created by Justin Douty on 7/8/22.

#define TUNING 264
// An arbitrary number specifying the "tuning" of the virtual instruments. Use this to tune the music to the correct pitch.

#define DECAY 0.50
// The decay of the notes after they stop playing, in seconds.

#include <iostream>
#include <cmath>
#include <string.h>
#include <chrono>
#include "MidiFile/MidiFile.h" // Midi parsing library https://github.com/craigsapp/midifile
#include "WavHeader.h"
#include "CUDAInstLoader.h"

using namespace std;
using namespace smf;

// Note GPU Kernel
// audioData: large allocation for the audio results of the entire song.
// start: where this kernel should start writing to in audioData; start of the note
// end: end of the note
// decayEnd: end of the note's decay; where kernel stops writing
// instSection: the instrument audio data (end is already constrained to prevent going off the end of the instrument section)
__global__ void processNote(float* audioData, unsigned long start, unsigned long end, unsigned long decayEnd, InstSection instSection, float pitchMultiplier, float velocityMultiplier) {
  unsigned long id = threadIdx.x + blockIdx.x * blockDim.x; 
  if(id < start)
    return;
  if(id+start >= decayEnd)
    return;

  ////////////////////////////
  //TODO: update audioData[id]
  ////////////////////////////


  unsigned long instOffset = (unsigned long) ((id-start) * pitchMultiplier);
  audioData[id] += instSection.data[instOffset] * velocityMultiplier;  
   
  unsigned long noteSize = decayEnd - end;
  float releaseVol = (float) (decayEnd - id) / (float) noteSize;
  audioData[id] += instSection.data[instOffset] * velocityMultiplier * releaseVol;


}

// Helper error-checking function for kernel launches
void checkErrors(string kernName, bool* debugToken, int blocks, int threads) {
  if(*debugToken) cudaDeviceSynchronize();
    auto cuerr = cudaGetLastError(); // NOTE: "This may return error codes from previous, asynchronous launches" - https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
  if(cuerr != 0) { 
    printf("%s<<<%d, %d>>>: %s: %s\n", kernName.c_str(), blocks, threads, cudaGetErrorName(cuerr), cudaGetErrorString(cuerr));
    *debugToken = true;
  }
}

int main(int argc, const char * argv[]) {
  std::setbuf(stdout, nullptr); // Disable output buffering
  std::setbuf(stderr, nullptr);
  
  string midFile = "symphony_5_1_(c)cvikl.mid"; // Specify the name of the MIDI file here

  // Define instrument audio data paths
  string instDir = "InstrumentData";
  /* Specify the order and files of instruments to use here... */
  string instDataNames[16] = {"Flute.wav", "Oboe.wav", "Clarinet.wav", "Bassoon.wav", "FrenchHorn.wav", "Trumpet.wav", "Timpani.wav", "Cello.wav", "Cello.wav", "Cello.wav", "Cello.wav", "Strings.wav", "Violin.wav", "Strings.wav", "Cello.wav", "Contrabass.wav"};

  /* ===== Load the instrument data files into memory. ===== */
  CUDAInstLoader lab(instDir, instDataNames);

  // Open and read in the midi file
  MidiFile mFile;
  if(! mFile.read(midFile)) {
    cerr << "Error opening input MIDI file!\n";
    exit(1);
  }
  mFile.joinTracks(); // Combine all event scheduling into a single track
  mFile.doTimeAnalysis(); // Finds the absolute starting points of all notes
  mFile.linkNotePairs(); // Matches up note-on events to note-off events, and finds the time in-between.
  
  auto start_time = std::chrono::high_resolution_clock::now(); // Measure start time
  
  // Create, and zero final audio data (this will be the contents of the entire resulting WAVE file...)
  int totalSteps = mFile.getFileDurationInSeconds() * 44100 + DECAY*44100; // The number of audio samples across the the total length of the WAVE data (as floats).
  float* audioData_dev = nullptr;
  if(cudaMalloc((void**) &audioData_dev, totalSteps * sizeof(float)) == cudaErrorMemoryAllocation) // Resulting audio data
    printf("Error: not enough memory to allocate audio data on device\n");
  cudaMemset((void*) audioData_dev, 0, totalSteps * sizeof(float)); // Clear to zero

  /* Audio Generation Section */
  int trackSize = mFile[0].size(); // The total number of events in the song
  bool debugMode = false; // Set to true to synchronize the kernels to get more reliable error messages (slower)
    
  for(int event = 0; event < trackSize; ++event) {
    //cout << event << " / " << trackSize << "\n";
    MidiEvent* e = &mFile[0][event]; // Current MIDI event.
    if(! e->isNoteOn()) continue; // We only need to use note-on events and their duration.

    InstSection section = lab.getInstSection(e->getChannel());
    
    // Converts MIDI note numbers into a pitch stretch factor.
    float pitchMultiplier = (440.0f*pow(2.0f, ((e->getKeyNumber()-69)/12.0f)) / (float) TUNING);
    float velocityMultiplier = (e->getVelocity()/127.0); // Convert 0-127 velocity into 0-1
    
    // start < end < decayEnd
    unsigned long start = e->seconds * 44100;
    unsigned long end = min( (unsigned long) (start + e->getDurationInSeconds() * 44100), (unsigned long) (start + section.size / pitchMultiplier) );
    
    unsigned long decayEnd = min( (unsigned long) (end + DECAY*44100), (unsigned long) (start + section.size / pitchMultiplier));
    
    // Find threads and blocks for note generation kernel
    unsigned long noteSize = decayEnd;
    if(noteSize == 0) continue;
    int threads, blocks;
    threads = 32; 
    blocks = (noteSize+threads-1) / threads;

    processNote<<<blocks, threads>>>(audioData_dev, start, end, decayEnd, section, pitchMultiplier, velocityMultiplier);
    checkErrors("Note Kernel", &debugMode, blocks, threads);
  }
  
  /* Copy audio data back from GPU */
  float* audioData = new float[totalSteps];
  cudaMemcpy(audioData, audioData_dev, sizeof(float) * totalSteps, cudaMemcpyDeviceToHost);

  auto end_time = std::chrono::high_resolution_clock::now();
  cerr << "EXECUTION TIME: " << std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time).count() << " milliseconds\n";
  
  lab.writeWaveFile(audioData, totalSteps, "out_cuda.wav");
}


In [ ]:
# Run this cell to play a portion of the CUDA generated audio file.
import numpy as np
import IPython.display as ipd

# You can set the start and end percentages of the song to play, or download the file from Colab and listen
start = 0
end = 5

a = np.fromfile("out_cuda.wav", dtype=np.float32, offset=90+4*start, sep="") # Read data from output file
startPos = int( 90 + (start / 100) * (len(a)-90) )
endPos = int( 90 + (end / 100) * (len(a)-90) )

ipd.Audio(a[startPos : endPos], rate=44100, autoplay = True) # Download and play audio